In [ ]:
import bloqade.stim
import bloqade.tsim

import warnings

import cirq
import numpy as np
import matplotlib.pyplot as plt
import bloqade.cirq_utils as utils
from cirq.contrib.svg import SVGCircuit

from bloqade import squin, cirq_utils

warnings.filterwarnings("ignore")

@squin.kernel
def main():
    q_data = squin.qalloc(7)
    # squin.u3(theta, phi, 0, q_data[6])

    # apply sqrt_y_adj to first 5 gates
    for i in range(6):
        squin.sqrt_y_adj(q_data[i])

    # apply cz to pairs
    squin.cz(q_data[1], q_data[2])
    squin.cz(q_data[3], q_data[4])
    squin.cz(q_data[5], q_data[6])

    # apply sqrt_y gate to injection state
    squin.sqrt_y(q_data[6])

    # apply cz to more pairs
    squin.cz(q_data[0], q_data[3])
    squin.cz(q_data[2], q_data[5])
    squin.cz(q_data[4], q_data[6])

    # apply sqrt_y gates
    for i in range(2, 7):
        squin.sqrt_y(q_data[i])

    # apply MORE cz gates
    squin.cz(q_data[0], q_data[1])
    squin.cz(q_data[2], q_data[3])
    squin.cz(q_data[4], q_data[5])

    squin.sqrt_y(q_data[1])
    squin.sqrt_y(q_data[2])
    squin.sqrt_y(q_data[4])

    """
    Encode auxillatry qbit
    """
    q_aux = squin.qalloc(7)
    squin.h(q_aux[6])

    # apply sqrt_y_adj to first 5 gates
    for i in range(6):
        squin.sqrt_y_adj(q_aux[i])

    # apply cz to pairs
    squin.cz(q_aux[1], q_aux[2])
    squin.cz(q_aux[3], q_aux[4])
    squin.cz(q_aux[5], q_aux[6])

    # apply sqrt_y gate to injection state
    squin.sqrt_y(q_aux[6])

    # apply cz to more pairs
    squin.cz(q_aux[0], q_aux[3])
    squin.cz(q_aux[2], q_aux[5])
    squin.cz(q_aux[4], q_aux[6])

    # apply sqrt_y gates
    for i in range(2, 7):
        squin.sqrt_y(q_aux[i])

    # apply MORE cz gates
    squin.cz(q_aux[0], q_aux[1])
    squin.cz(q_aux[2], q_aux[3])
    squin.cz(q_aux[4], q_aux[5])

    squin.sqrt_y(q_aux[1])
    squin.sqrt_y(q_aux[2])
    squin.sqrt_y(q_aux[4])

    """
    Manual bit flip
    X-gate applied to 6th qbit of q_data

    Then couple with CNOT gate to data and auxillary logical qbit 
    """
    squin.x(q_data[5])
    for i in range(7):
        squin.cx(q_data[i], q_aux[i])

    # introduce error by bit-flipping 5th physical qbit in 
    squin.broadcast.measure(q_aux, key='result')

# plotting circuit diagram (plotting is nicer on Tsim)
tsim_circ = bloqade.tsim.Circuit(main)
tsim_circ.diagram(height=400)

# sampling using Stim (Clifford-only sampling is faster on Stim)
# stim_circ = bloqade.stim.Circuit(main)
# sampler = stim_circ.compile_sampler()
# samples = sampler.sample(shots=5)
# print(samples)